In [78]:
import fastf1
import pandas as pd
import numpy as np

In [79]:
def get_driver_data(driver):
    session = fastf1.get_session(2024, 'Silverstone', 'R')
    session.load(telemetry=True) 

    driver_general_data = pd.DataFrame(session.laps.pick_driver(driver))

    driver_data = session.laps.pick_driver(driver)
    return driver_data
    

In [80]:
def adjust_laptimes(dataframe):
    dataframe["Laptime"] = dataframe["LapTime"].astype(int) * 1e-9  # Converts LapTime to seconds
    dataframe["Position(s)"] = dataframe["Position"].astype(int)  # Converts Position to integer
    new_df = dataframe.drop(columns=["LapTime", "Position"])  # Drops LapTime and Position columns
    new_df = new_df.rename(columns={"Position(s)": "Position"})  # Renames Position(s) to Position
    return new_df

def get_common(dataframe):
    new_df = dataframe[["LapNumber", "TrackStatus"]]  # Selects only LapNumber and TrackStatus
    return new_df

def get_pitstop(dataframe):
    # Replace NaT or NaN with 0 for both columns
    dataframe["PitInTime"] = dataframe["PitInTime"].fillna(pd.Timedelta(0))
    dataframe["PitOutTime"] = dataframe["PitOutTime"].fillna(pd.Timedelta(0))
    
    # Convert Timedelta to seconds using total_seconds()
    dataframe["PitInTime"] = dataframe["PitInTime"].apply(lambda x: x.total_seconds() if isinstance(x, pd.Timedelta) else x)
    dataframe["PitOutTime"] = dataframe["PitOutTime"].apply(lambda x: x.total_seconds() if isinstance(x, pd.Timedelta) else x)
    
    # Create the PitstopTime column as the difference between PitOutTime and PitInTime
    dataframe["PitstopTime"] = dataframe["PitOutTime"] - dataframe["PitInTime"]
    
    return dataframe


# def adjust_pitstop_times(dataframe):
    

In [81]:
def remove_extras(dataframe):
    new_df = dataframe[["Driver", "Laptime", "PitstopTime", "Position", "Compound", "Position", "Source"]]  # Drops unnecessary columns
    return new_df


In [82]:
def drop_unecessary_telem(dataframe):
    dropped_dataframe = dataframe.drop(columns=["DriverAhead", 'SessionTime', 'Date', 'Time'])
    guess_free_dataframe = dropped_dataframe[dropped_dataframe['Source'] != 'interpolation']
    return guess_free_dataframe

In [83]:
driver_telemetry = pd.DataFrame()

bot_data = get_driver_data('BOT')


core           INFO 	Loading data for British Grand Prix - Race [v3.4.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2024/12/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\smmey\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\smmey\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\smmey\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of th

In [84]:
bot_data

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
726,0 days 01:00:00.089000,BOT,77,0 days 00:01:44.290000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:40.380000,...,True,Kick Sauber,0 days 00:58:15.512000,2024-07-07 14:03:12.541,1,19.0,False,,False,False
727,0 days 01:01:35.457000,BOT,77,0 days 00:01:35.368000,2.0,1.0,NaT,NaT,0 days 00:00:30.060000,0 days 00:00:39.194000,...,True,Kick Sauber,0 days 01:00:00.089000,2024-07-07 14:04:57.118,1,18.0,False,,False,True
728,0 days 01:03:09.904000,BOT,77,0 days 00:01:34.447000,3.0,1.0,NaT,NaT,0 days 00:00:30.096000,0 days 00:00:38.414000,...,True,Kick Sauber,0 days 01:01:35.457000,2024-07-07 14:06:32.486,1,18.0,False,,False,True
729,0 days 01:04:44.226000,BOT,77,0 days 00:01:34.322000,4.0,1.0,NaT,NaT,0 days 00:00:29.890000,0 days 00:00:38.839000,...,True,Kick Sauber,0 days 01:03:09.904000,2024-07-07 14:08:06.933,1,19.0,False,,False,True
730,0 days 01:06:17.994000,BOT,77,0 days 00:01:33.768000,5.0,1.0,NaT,NaT,0 days 00:00:29.920000,0 days 00:00:38.115000,...,True,Kick Sauber,0 days 01:04:44.226000,2024-07-07 14:09:41.255,1,19.0,False,,False,True
731,0 days 01:07:51.454000,BOT,77,0 days 00:01:33.460000,6.0,1.0,NaT,NaT,0 days 00:00:29.755000,0 days 00:00:37.957000,...,True,Kick Sauber,0 days 01:06:17.994000,2024-07-07 14:11:15.023,1,19.0,False,,False,True
732,0 days 01:09:25.102000,BOT,77,0 days 00:01:33.648000,7.0,1.0,NaT,NaT,0 days 00:00:29.919000,0 days 00:00:37.898000,...,True,Kick Sauber,0 days 01:07:51.454000,2024-07-07 14:12:48.483,1,19.0,False,,False,True
733,0 days 01:10:58.663000,BOT,77,0 days 00:01:33.561000,8.0,1.0,NaT,NaT,0 days 00:00:29.899000,0 days 00:00:37.968000,...,True,Kick Sauber,0 days 01:09:25.102000,2024-07-07 14:14:22.131,1,19.0,False,,False,True
734,0 days 01:12:32.308000,BOT,77,0 days 00:01:33.645000,9.0,1.0,NaT,NaT,0 days 00:00:29.917000,0 days 00:00:37.913000,...,True,Kick Sauber,0 days 01:10:58.663000,2024-07-07 14:15:55.692,1,19.0,False,,False,True
735,0 days 01:14:06.095000,BOT,77,0 days 00:01:33.787000,10.0,1.0,NaT,NaT,0 days 00:00:29.850000,0 days 00:00:38.666000,...,True,Kick Sauber,0 days 01:12:32.308000,2024-07-07 14:17:29.337,1,18.0,False,,False,True


In [85]:
for lap_number, lap in bot_data.iterlaps():
    telemetry = lap.get_telemetry()
    telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'LapTime']]

    driver_telemetry = pd.concat([driver_telemetry, telemetry])

driver_telemetry

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
C:\Users\smmey\AppData\Local\Temp\ipykernel_23408\2697744677.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'LapTime']]
C:\Users\smmey\AppData\Local\Temp\ipykernel_23408\2697744677.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'Lap

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
2,2024-07-07 14:03:12.541,0 days 00:58:15.512000,,0.088889,0 days 00:00:00,10543,0,1,23,False,1,interpolation,0.019699,0.000003,OnTrack,-1768.914203,1192.129451,1963.000018,1.0,0 days 00:01:44.290000
3,2024-07-07 14:03:12.660,0 days 00:58:15.631000,,0.088889,0 days 00:00:00.119000,10539,0,1,23,False,1,car,0.000000,0.000000,OnTrack,-1768.894531,1192.159145,1963.000023,1.0,0 days 00:01:44.290000
4,2024-07-07 14:03:12.831,0 days 00:58:15.802000,,0.088889,0 days 00:00:00.290000,10282,0,1,23,False,1,pos,-0.069833,-0.000012,OnTrack,-1769.000000,1192.000000,1963.000000,1.0,0 days 00:01:44.290000
5,2024-07-07 14:03:12.931,0 days 00:58:15.902000,,0.088889,0 days 00:00:00.390000,10025,0,1,23,False,1,pos,-0.052149,-0.000009,OnTrack,-1769.000000,1192.000000,1963.000000,1.0,0 days 00:01:44.290000
6,2024-07-07 14:03:12.980,0 days 00:58:15.951000,14,0.088889,0 days 00:00:00.439000,9768,0,1,24,False,1,car,0.000000,0.000000,OnTrack,-1768.692369,1192.464191,1963.000066,1.0,0 days 00:01:44.290000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,2024-07-07 15:26:21.333,0 days 02:21:24.304000,20,2481.290556,0 days 00:01:31.842000,11515,249,6,100,False,0,pos,5791.307526,0.991412,OnTrack,-1894.000000,1022.000000,1963.000000,51.0,0 days 00:01:32.217000
708,2024-07-07 15:26:21.516,0 days 02:21:24.487000,20,2481.407222,0 days 00:01:32.025000,11597,250,6,100,False,0,car,5804.041667,0.993592,OnTrack,-1816.087263,1128.527684,1963.000556,51.0,0 days 00:01:32.217000
709,2024-07-07 15:26:21.573,0 days 02:21:24.544000,20,2481.540556,0 days 00:01:32.082000,11177,252,6,100,False,0,pos,5808.058218,0.994279,OnTrack,-1792.000000,1161.000000,1963.000000,51.0,0 days 00:01:32.217000
710,2024-07-07 15:26:21.676,0 days 02:21:24.647000,20,2481.673889,0 days 00:01:32.185000,10757,255,6,100,False,0,car,5815.375000,0.995532,OnTrack,-1748.218770,1219.625900,1963.154270,51.0,0 days 00:01:32.217000


In [86]:
driver_telemetry = pd.DataFrame(drop_unecessary_telem(driver_telemetry))

In [87]:
def adjust_laptimes(dataframe: pd.DataFrame):
    dataframe["Laptime"] = dataframe["Laptime"].astype("int64") * 1e-9  # Converts LapTime to seconds # Drops LapTime and Position columns
    return dataframe

In [88]:
driver_telemetry = adjust_laptimes(driver_telemetry)

### Min Max and Normalize

In [89]:
from sklearn.preprocessing import MinMaxScaler

In [90]:
scaler = MinMaxScaler()

normalize_columns = ['RPM', 'DistanceToDriverAhead', 'Speed', 'Distance', 'X', 'Y', 'Z', 'RPM', 'Throttle', 'RelativeDistance', 'Laptime']


driver_telemetry[normalize_columns] = scaler.fit_transform(pd.DataFrame(driver_telemetry[normalize_columns]))



In [91]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
3,0.000036,0.779012,0.000000,1,0.23,False,1,car,0.000012,0.000012,OnTrack,0.054515,0.308233,0.017485,1.0,0.368108
4,0.000036,0.751825,0.000000,1,0.23,False,1,pos,0.000000,0.000000,OnTrack,0.054505,0.308224,0.017485,1.0,0.368108
5,0.000036,0.724638,0.000000,1,0.23,False,1,pos,0.000003,0.000003,OnTrack,0.054505,0.308224,0.017485,1.0,0.368108
6,0.000036,0.697451,0.000000,1,0.24,False,1,car,0.000012,0.000012,OnTrack,0.054535,0.308250,0.017486,1.0,0.368108
7,0.000018,0.549349,0.018576,1,0.24,False,1,pos,0.000119,0.000119,OnTrack,0.054900,0.308572,0.017485,1.0,0.368108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0.999799,0.873691,0.767802,6,1.00,False,0,car,0.987195,0.990630,OnTrack,0.037893,0.294869,0.013389,51.0,0.026590
707,0.999846,0.882260,0.770898,6,1.00,False,0,pos,0.988340,0.991779,OnTrack,0.042150,0.298364,0.017485,51.0,0.026590
708,0.999893,0.890934,0.773994,6,1.00,False,0,car,0.990513,0.993959,OnTrack,0.049851,0.304542,0.017490,51.0,0.026590
709,0.999946,0.846504,0.780186,6,1.00,False,0,pos,0.991198,0.994647,OnTrack,0.052232,0.306426,0.017485,51.0,0.026590


In [92]:
driver_telemetry['Brake'] = driver_telemetry['Brake'].astype(int)

In [93]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
3,0.000036,0.779012,0.000000,1,0.23,0,1,car,0.000012,0.000012,OnTrack,0.054515,0.308233,0.017485,1.0,0.368108
4,0.000036,0.751825,0.000000,1,0.23,0,1,pos,0.000000,0.000000,OnTrack,0.054505,0.308224,0.017485,1.0,0.368108
5,0.000036,0.724638,0.000000,1,0.23,0,1,pos,0.000003,0.000003,OnTrack,0.054505,0.308224,0.017485,1.0,0.368108
6,0.000036,0.697451,0.000000,1,0.24,0,1,car,0.000012,0.000012,OnTrack,0.054535,0.308250,0.017486,1.0,0.368108
7,0.000018,0.549349,0.018576,1,0.24,0,1,pos,0.000119,0.000119,OnTrack,0.054900,0.308572,0.017485,1.0,0.368108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0.999799,0.873691,0.767802,6,1.00,0,0,car,0.987195,0.990630,OnTrack,0.037893,0.294869,0.013389,51.0,0.026590
707,0.999846,0.882260,0.770898,6,1.00,0,0,pos,0.988340,0.991779,OnTrack,0.042150,0.298364,0.017485,51.0,0.026590
708,0.999893,0.890934,0.773994,6,1.00,0,0,car,0.990513,0.993959,OnTrack,0.049851,0.304542,0.017490,51.0,0.026590
709,0.999946,0.846504,0.780186,6,1.00,0,0,pos,0.991198,0.994647,OnTrack,0.052232,0.306426,0.017485,51.0,0.026590


In [94]:
lap_times_label = driver_telemetry.groupby('LapNumber')['Laptime'].max().reset_index()

In [95]:
lap_times_label

,LapNumber,Laptime
0,1.0,0.368108
1,2.0,0.115725
2,3.0,0.089672
3,4.0,0.086136
4,5.0,0.070465
5,6.0,0.061752
6,7.0,0.067070
7,8.0,0.064609
8,9.0,0.066985
9,10.0,0.071002


In [96]:
pad_max = (0,0)
avg = 0
for x in range(1,52):
    curr = driver_telemetry[driver_telemetry['LapNumber'] == x].shape[0]
    
    avg += curr
    if curr > pad_max[1]:
        pad_max = (x, curr)
print("Most telemetry points in 1 lap:", pad_max)
print("Average Telemetry Points per Lap:", avg // 51)

Most telemetry points in 1 lap: (38, 952)
Average Telemetry Points per Lap: 746


In [97]:
driver_telemetry = driver_telemetry.drop(columns=['Laptime'])

In [98]:
padded_data = []
for x in range(1,52):
    lap_data = driver_telemetry[driver_telemetry['LapNumber'] == x]
    telem_points = driver_telemetry[driver_telemetry['LapNumber'] == x].shape[0]
    if telem_points < pad_max[1]:
           padding = pd.DataFrame(np.zeros((pad_max[1] - telem_points, driver_telemetry.shape[1])), columns= driver_telemetry.columns)
           padding['LapNumber'] = x
           lap_data = pd.concat([driver_telemetry[driver_telemetry['LapNumber'] == x], padding])
    padded_data.append(lap_data)

driver_telemetry = pd.concat(padded_data, ignore_index= True)

In [99]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber
0,0.000036,0.779012,0.000000,1.0,0.23,0.0,1.0,car,0.000012,0.000012,OnTrack,0.054515,0.308233,0.017485,1.0
1,0.000036,0.751825,0.000000,1.0,0.23,0.0,1.0,pos,0.000000,0.000000,OnTrack,0.054505,0.308224,0.017485,1.0
2,0.000036,0.724638,0.000000,1.0,0.23,0.0,1.0,pos,0.000003,0.000003,OnTrack,0.054505,0.308224,0.017485,1.0
3,0.000036,0.697451,0.000000,1.0,0.24,0.0,1.0,car,0.000012,0.000012,OnTrack,0.054535,0.308250,0.017486,1.0
4,0.000018,0.549349,0.018576,1.0,0.24,0.0,1.0,pos,0.000119,0.000119,OnTrack,0.054900,0.308572,0.017485,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48547,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,51.0
48548,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,51.0
48549,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,51.0
48550,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,51.0
